# Face identification problem

**Task**
Challenge yourself with image classification task for solving face identification problem on ORL dataset available here.

**Goal**
The aim of this project is listed (but not limited to) below:
    - Familiarize yourself with EDA (Exploratory Data Analysis) technique on the concrete example with dataset of people’ faces.
    - Implement simplest classification algorithm with Python language.
    - Investigate general techniques of normalization and optimization.
    - Experience yourself in report with detailed explanation of applied strategy and the achieved results.
You can familiarize yourself with the common methods in CV (Computer Vision), image processing, and face recognition on the example here.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
# import h5py
# import scipy
# from PIL import Image
# from scipy import ndimage

%matplotlib inline

In [16]:
# model const & parameters 
faces_folder = 'att_faces/'
person_count = 40
faces_count = 10
train_percent = 80.0

faces_count_train = round (faces_count * train_percent / 100)
faces_count_verif = faces_count - faces_count_train
print ("Train faces count = " + str(faces_count_train))
print ("Verif faces count = " + str(faces_count_verif))

Train faces count = 8
Verif faces count = 2


In [18]:
def load_datasets ():
    #     TODO: implement the function
    
    pass

   

In [19]:
# Loading the data (faces)
load_datasets ()